In [1]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

from data_integration import combine_data, read_data
# needed to read csv faster
import polars as pl 

## combine all

In [ ]:
"""
    Calls read_data() from file "data_integration.py"
    Calls the combine_data() function from file "data_integration.py"
    Parameter:
    - 1: combine only glucose data
    - 2: combine glucose and dempgraphics
    - 3: combine glucose and heartrate
    Output: This function returns the intrgrated dataset included the restricted databases if downloaded and added to folder "datasets for T1D"
 """
restricted_list = read_data(read_all = False)

# load the main dataset and combine it with the restricted data
combined_main_list = pl.read_csv("datasets for T1D/maindatabase.csv").to_pandas()
combined_main = combine_data(1, restricted_list)  
combined_main = pd.concat([combined_main, combined_main_list])
# save the full integrated dataset as a csv file
combined_main.to_csv("datasets for T1D/fullmaindatabase.csv", index=False)

In [ ]:
# restricted_list = read_data(read_all = False)

# load the subdatasetI and combine it with the restricted data
combined_subI_list =  pl.read_csv("datasets for T1D/subdatabaseI.csv").to_pandas()
combined_subI = combine_data(2, restricted_list)
combined_subI = pd.concat([combined_subI, combined_subI_list])
# save the full integrated dataset as a csv file
combined_subI.to_csv("datasets for T1D/fullsubdatabaseI.csv", index=False)

In [ ]:
# load the subdatasetII, no need to concatenate since the restricted databases do not include HR
combined_subII_list =  pl.read_csv("datasets for T1D/subdatabaseII.csv").to_pandas()

## Figures

### Glucose Characteristics

In [ ]:
# remove null values in the GLucoseCGM column in the Main Database 
combined_main_filtered = combined_main.dropna(subset=["GlucoseCGM"])

# assign euglycemic, hyperglycemic, and hyperglycemic values 
combined_main_filtered["GLCRange"] = "Euglycemia"
combined_main_filtered.loc[combined_main_filtered["GlucoseCGM"] <= 70, "GLCRange"] = "Hypoglycemia"
combined_main_filtered.loc[combined_main_filtered["GlucoseCGM"] >= 180, "GLCRange"] = "Hyperglycemia"

ranges = combined_main_filtered["GLCRange"].unique()

# copy the dataframe and assign level 1 and level 2 hypogylcemic values
combined_plus_severe = combined_main_filtered.copy()
combined_plus_severe.loc[combined_plus_severe["GlucoseCGM"] <= 54, "GLCRange"] = "Severe Hypoglycemia"
filtered_glucose_hypo = combined_plus_severe[(combined_plus_severe["GLCRange"] == "Hypoglycemia") | (combined_plus_severe["GLCRange"] == "Severe Hypoglycemia")]
glucose_hypo_counts = filtered_glucose_hypo["GLCRange"].value_counts().sort_index()

# count the glucose values per glucose range categrory 
glucose_char_counts = combined_main_filtered["GLCRange"].value_counts().sort_index()

# Function to show both value and percentage
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct * total / 100.0))
        val_mio = round(val / 1_000_000, 1)
        return f"{val} ({pct:.1f}%)"
    return my_autopct


# plot the distribution of euglycemia, hypoglycemia, hyperglycemia, and plot the distribution of hypoglycemia and severe hypoglycemia
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.pie(glucose_char_counts, labels=glucose_char_counts.index.astype(str), autopct=make_autopct(glucose_char_counts), startangle=140)
ax1.set_title("Total Glucose Values Distribution")

# Bar chart
ax2.bar(glucose_hypo_counts.index.astype(str), glucose_hypo_counts.values, width=0.8, edgecolor="black", color = ["green", "green"])
ax2.set_title("Hypoglycemic Values")
ax2.set_ylabel("Count")

# Formatter function
def millions(x, pos):
    return f"{x*1e-6:.0f} Mio"

# Apply formatter function to the axis to shorten to Mio 
ax2.yaxis.set_major_formatter(FuncFormatter(millions))


# Add text labels on top of each bar
for bar in ax2.patches:
    height = bar.get_height()
    label = f"{height / 1e6:.2f} Mio".rstrip("0").rstrip(".")  # Converts to millions, cleans up trailing zeros
    ax2.text(bar.get_x() + bar.get_width() / 2, height, label,
             ha="center", va="bottom")

# Layout
plt.tight_layout()
plt.show()
plt.show()

In [ ]:
# Plot boxplot
plt.figure(figsize=(8,6))
# drop nan values in glucose
combined_df_g_filtered = combined_main.dropna(subset=["GlucoseCGM"])
# plot the glucose variation among databases
combined_df_g_filtered.boxplot(column="GlucoseCGM", by="Database", grid=True,showmeans=True)
plt.xticks(rotation=90)
plt.title("Glucose Variation by Database")
plt.suptitle("") 
plt.xlabel("Database")
plt.ylabel("Glucose")
plt.show()

### Exploring Missing Values 

In [ ]:
# Plot missing values to assess data quality
# Function to show both value and percentage
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct * total / 100.0))
        val_mio = round(val / 1_000_000, 1)
        return f"{val} ({pct:.1f}%)"
    return my_autopct

# Identify missing blocks
df_glc_missing = combined_main.copy()
df_glc_missing["missing"] = df_glc_missing["GlucoseCGM"].isna()


# use groupby with cumsum to find continuous blocks
df_glc_missing["gap_id"] = (df_glc_missing["missing"] != df_glc_missing["missing"].shift()).cumsum()
# group by the identified gaps
na_groups = df_glc_missing[df_glc_missing["missing"]].groupby("gap_id").size().reset_index(name="missing_count")
# one row of missing data equals 5 minutes, so the length of gap is multiplied by 5
na_groups["missing_count_time"] = na_groups["missing_count"] * 5

# Categorize durations
bins = [0,30,60,120,240,1440, na_groups["missing_count_time"].max()]
labels = [
    "<30 min",
    "30 min-1 h",
    "1-2 h",
    "2-4 h",
    "4-24 h",
    ">24 h"
]

# Convert durations to timedelta seconds for binning
duration_bins = pd.cut(na_groups["missing_count_time"], bins=bins, labels=labels, right=False)

# Count number of gaps per duration range
gap_counts = duration_bins.value_counts().sort_index()

# create the bin groups
na_groups["MissingGroup"] = pd.cut(na_groups["missing_count_time"], bins=bins, labels=labels, right=False)
# filter to only less than 30 minute gaps
count_30_missing = na_groups[na_groups["MissingGroup"] ==  "<30 min"]["missing_count_time"].value_counts()

# plot the missing values count per created group and for the second group of less than 30 minutes in 5 minute intervals
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.bar(gap_counts.index.astype(str), gap_counts.values)
ax1.set_title("Missing Data Gaps by Duration")
ax1.set_xlabel("Duration of Gaps")
ax1.set_ylabel("Number of Gaps")

# Formatter function
def millions(x, pos):
    return f"{x*1e-6:.2f} Mio"

# Apply formatter to axis
ax1.yaxis.set_major_formatter(FuncFormatter(millions))
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90)

# plot the numbers on the bar
for bar in ax1.patches:
    height = bar.get_height()
    label = f"{height / 1e6:.2f} Mio".rstrip("0").rstrip(".")  # Converts to millions, cleans up trailing zeros
    ax1.text(bar.get_x() + bar.get_width() / 2, height, label,
             ha="center", va="bottom")

# Bar chart
ax2.bar(count_30_missing.index.astype(str), count_30_missing.values, width=0.8, edgecolor="black")
ax2.set_title("Gaps Less Than 30 Min")
ax2.set_xlabel("Duration of Gaps")
ax2.set_ylabel("Number of Gaps")

# Add text labels on top of each bar
for bar in ax2.patches:
    height = bar.get_height()
    label = f"{height / 1e6:.2f} Mio".rstrip("0").rstrip(".")  # Converts to millions, cleans up trailing zeros
    ax2.text(bar.get_x() + bar.get_width() / 2, height, label,
             ha="center", va="bottom")

# Apply formatter to axis 
ax2.yaxis.set_major_formatter(FuncFormatter(millions))
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=90)

# Layout
plt.tight_layout()
plt.show()


In [ ]:
# extract the database name 
df_glc = combined_main.copy()
# group per database and number of missing values in total
df_missing_counts = df_glc["GlucoseCGM"].isnull().groupby(df_glc["Database"]).sum()
df_missing_counts

In [ ]:
# plot missing values per single database
bars = plt.bar(df_missing_counts.index.astype(str), df_missing_counts.values)

plt.gca().set_yticklabels([f'{int(tick/1_000_000)} Mio' for tick in plt.gca().get_yticks()])

plt.xticks(rotation=90)
plt.title("Missing Data Gaps per Individual Database")
plt.xlabel("Database")
plt.ylabel("Number of Gaps")
plt.tight_layout()
plt.show()

### Analysis of Demographics

In [ ]:
# Plot the Distribution of Age groups across Subdataset I
# first copy the dataset
df = combined_subI.copy()
# then extract each PtID and the corresponsing AgeGroup
df_demo_only_ages = df.drop_duplicates(subset="PtID")[["PtID", "AgeGroup"]]
# count the distributions of agegroups
age_counts = df_demo_only_ages["AgeGroup"].value_counts().sort_index()
total = age_counts.sum()
# calculate the percentage for plotting
percentages = (age_counts / total * 100).round(1)


# Plot the bar plot of grouped ages
plt.figure(figsize=(8, 5))
bars = plt.bar(age_counts.index.astype(str), age_counts.values, width=0.8, edgecolor="black")
plt.title("Distribution of Age Groups Across the Dataset")
plt.xlabel("Age Group")
plt.ylabel("Number of Subjects")

# Add percentage labels
for bar, pct in zip(bars, percentages):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 1, f"{pct}%", 
                ha="center", va="bottom", fontsize=9)


plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Plot histogram of all ages and the number of glucose measurements
# first remove null values of GlucoseCGM
combined_subI2_filtered = combined_subI.dropna(subset=["GlucoseCGM"])
# Count number of patients in each group
glucoseage_counts = combined_subI2_filtered["AgeGroup"].value_counts().sort_index()
total = glucoseage_counts.sum()
# calculate percentage for plotting
percentages = (glucoseage_counts / total * 100).round(1)

# Plot the figure
plt.figure(figsize=(8, 5))
plt.bar(glucoseage_counts.index.astype(str), glucoseage_counts.values, width=0.8, edgecolor="black")
plt.title("Measurement Frequency by Age Groups")
plt.xlabel("Age Group")
plt.ylabel("Number of Glucose Measurements")

# Add percentage labels
for bar, pct in zip(bars, percentages):
    height = bar.get_height()
    if height > 0:
        plt.text(bar.get_x() + bar.get_width()/2, height + 1, f"{pct}%", 
                    ha="center", va="bottom", fontsize=9)

# shorten the number into mio  
plt.gca().set_yticklabels([f"{int(tick/1_000_000)} Mio" for tick in plt.gca().get_yticks()])
plt.tight_layout()
plt.show()

In [ ]:
# Plot the distribution of sex across Database I
# first extract the PtIDs and the corresponding sex
df_demo_only_sex = combined_subI.drop_duplicates(subset="PtID")[["PtID", "Sex"]]
# Count both Sexes
gender_counts = df_demo_only_sex["Sex"].value_counts().sort_index()
gender_counts.index.astype(str)

# gender related plotting 
total_gender = gender_counts.sum()
# calculate percentage for plotting
percentages_gender = (gender_counts / total_gender * 100).round(1)

# Function to show both value and percentage
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct * total / 100.0))
        val_mio = round(val / 1_000_000, 1)
        return f"{val} ({pct:.1f}%)"
    return my_autopct

# Plot
plt.figure(figsize=(6, 6))
plt.pie(gender_counts, labels=gender_counts.index.astype(str), autopct=make_autopct(gender_counts), startangle=140)
plt.title("Sex Distribution")
plt.axis("equal")  
plt.show()

In [ ]:
# Count number of values in each sex 
# take the prior filtered dataframe without null values in glucose and count values per sex
glucosegender_counts = combined_subI2_filtered["Sex"].value_counts().sort_index()

# plot
plt.figure(figsize=(6, 6))
plt.pie(glucosegender_counts, labels=glucosegender_counts.index.astype(str), autopct=make_autopct(glucosegender_counts), startangle=140)
plt.title("Glucose Values per Sex")
plt.axis("equal")  # Equal aspect ratio makes the pie chart circular.
plt.gca().set_yticklabels([f"{int(tick/1_000_000)} Mio" for tick in plt.gca().get_yticks()])
plt.show()


In [ ]:
# Extract the PtIDs and the corresponsing AgeGroups and Sex
df_demo_only = combined_subI.drop_duplicates(subset="PtID")[["PtID", "AgeGroup", "Sex"]]
# Count the number of Subjects per AgeGroups and per Sex in each AgeGroup
counts = df_demo_only.groupby(["AgeGroup", "Sex"]).size().unstack(fill_value=0)

# Plot
ax = counts.plot.bar(rot=0, color={"F": "orchid", "M": "steelblue"})
plt.title("Sex Count by Age Group")
plt.xlabel("Age Group")
plt.ylabel("Number of Subjects")
plt.legend(title="Sex")
plt.tight_layout()
plt.show()

In [ ]:
# take the prior filtered dataframe with no null values in the GlucoseCGm column
combined_subI2_filtered = combined_subI.dropna(subset=["GlucoseCGM"])
# count glucose values per AgeGroup and per Sex in each AgeGroup
counts_demo = combined_subI2_filtered.groupby(["AgeGroup", "Sex"]).size().unstack(fill_value=0)
# Plot stacked bar chart

# Plot
ax = counts_demo.plot.bar(rot=0, color={"F": "orchid", "M": "steelblue"}, )

plt.title("Measurement Frequency by Age Groups and Sex")
plt.xlabel("Age Group")
plt.ylabel("Number of Glucose Measurements")
plt.legend(title="Sex", loc="upper left")
plt.gca().set_yticklabels([f"{int(tick/1_000_000)} Mio" for tick in plt.gca().get_yticks()])
plt.tight_layout()
plt.show()

In [ ]:
# Plot boxplot of the variation of glucose levels across age groups
plt.figure(figsize=(8,6))
combined_subI2_filtered.boxplot(column="GlucoseCGM", by="AgeGroup", grid=True,showmeans=True)
plt.title("Glucose Levels by Age Group")
plt.suptitle("")  
plt.xlabel("Age Group")
plt.ylabel("Glucose Levels")
plt.show()

In [ ]:
# Plot boxplot of the variation of glucose levels among sex
plt.figure(figsize=(8,6))
combined_subI2_filtered.boxplot(column="GlucoseCGM", by="Sex", grid=True,showmeans=True)
plt.title("Glucose Levels by Sex")
plt.suptitle("")  
plt.xlabel("Sex")
plt.ylabel("Glucose Levels")
plt.show()

## Heart-rate Analysis

In [ ]:
# Plot boxplot for heartrate variation across single databases
plt.figure(figsize=(8,6))
combined_df_v_filtered = combined_subII.dropna(subset=["HR"])
combined_df_v_filtered.boxplot(column="HR", by="Database", grid=True,showmeans=True)
plt.title("Heart rate by Database")
plt.suptitle("")  
plt.xlabel("Database")
plt.ylabel("Heart rate")
plt.show()

In [ ]:
# Plot missing values of heartrate column
# Function to show both value and percentage
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct * total / 100.0))
        val_mio = round(val / 1_000_000, 1)
        return f"{val} ({pct:.1f}%)"
    return my_autopct

# Identify missing blocks in the heartrate colum
df_glc_missing = combined_subII.copy()
df_glc_missing["missing"] = df_glc_missing["HR"].isna()

# Use groupby with cumsum to find continuous blocks
df_glc_missing["gap_id"] = (df_glc_missing["missing"] != df_glc_missing["missing"].shift()).cumsum()
na_groups = df_glc_missing[df_glc_missing["missing"]].groupby("gap_id").size().reset_index(name="missing_count")
# multiply by 5 since one missing value euquals 5 minutes
na_groups["missing_count_time"] = na_groups["missing_count"] * 5

# Categorize durations
bins = [0,30,60,120,240,1440, na_groups["missing_count_time"].max()]
labels = [
    "<30 min",
    "30 min-1 h",
    "1-2 h",
    "2-4 h",
    "4-24 h",
    ">24 h"
]

# Convert durations to timedelta seconds for binning
duration_bins = pd.cut(na_groups["missing_count_time"], bins=bins, labels=labels, right=False)

# Count number of gaps per duration range
gap_counts = duration_bins.value_counts().sort_index()

# create bins as column and filter to less than 30 minute gaps
na_groups["MissingGroup"] = pd.cut(na_groups["missing_count_time"], bins=bins, labels=labels, right=False)
# plot only gaps less than 30 minutes in 5 minute intervals
count_30_missing = na_groups[na_groups["MissingGroup"] ==  "<30 min"]["missing_count_time"].value_counts()

# plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.bar(gap_counts.index.astype(str), gap_counts.values)
ax1.set_title("Missing Data Gaps by Duration for Heartrate")
ax1.set_xlabel("Duration of Gaps")
ax1.set_ylabel("Number of Gaps")
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90)

# plot the number oon the top of each bar
for bar in ax1.patches:
    height = bar.get_height()
    label = f'{height:.2f}'.rstrip('0').rstrip('.')  # Converts to millions, cleans up trailing zeros
    ax1.text(bar.get_x() + bar.get_width() / 2, height, label,
             ha='center', va='bottom')
    

# Bar chart
ax2.bar(count_30_missing.index.astype(str), count_30_missing.values, width=0.8, edgecolor="black")
ax2.set_title("Gaps Less Than 30 Min")
ax2.set_xlabel("Duration of Gaps")
ax2.set_ylabel("Number of Gaps")

# Add text labels on top of each bar
for bar in ax2.patches:
    height = bar.get_height()
    label = f'{height:.2f}'.rstrip('0').rstrip('.')  # Converts to millions, cleans up trailing zeros
    ax2.text(bar.get_x() + bar.get_width() / 2, height, label,
             ha='center', va='bottom')
    
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=90)

# Layout
plt.tight_layout()
plt.show()
